In [ ]:
#r "D:\BoSSS2\experimental\public\src\L4-application\BoSSSpad\bin\Release\net5.0\bossspad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSEC;
using NSE_SIMPLE;
using NSE_SIMPLE.LowMach;
using ilPSP.LinSolvers.PARDISO;

Init();

In [ ]:
using BoSSS.Solution.Queries;

In [ ]:
static string name = "CTD_SIMPLE_XNSEC_Comparison___final4";

In [ ]:
BoSSSshell.WorkflowMgm.Init(name);

In [ ]:
static var myBatch = ExecutionQueues[2];
static var myDb = myBatch.CreateOrOpenCompatibleDatabase(name);
// static var myDb =  BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.DefaultDatabase;


In [ ]:
BoSSSshell.WorkflowMgm.DefaultDatabase = myDb;

Opening existing database '\\hpccluster\hpccluster-scratch\gutierrez\CTD_SIMPLE_XNSEC_Comparison___final4'.


## Simple Controlfile

In [ ]:
myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(myDb.Path,""));

In [ ]:
// int[] dgDegree = new int[]{2,3};
// int[] res = new int[]{3};
int[] dgDegree = new int[]{1,2,3,4};
int[] res = new int[]{3,4};

Type solverSimple = typeof(NSE_SIMPLE.NSE_SIMPLEMain);
Type solverXNSEC = typeof(BoSSS.Application.XNSEC.XNSEC);



In [ ]:
// for (int dg = 0; dg < dgDegree.Length; dg++) {
//     for (int r = 0; r < res.Length; r++) {
//         string ending        = "P" + dgDegree[dg] + "K" + res[r];
//         string jobName       = string.Format("SIMPLE_HEATEDCOUETTE"+ ending);
//         string StartupString = string.Format("cs:NSE_SIMPLE.LowMach.ControlExamples.SteadyCouetteFlowWithTemperatureGradient({0}, {1},@\"{2}\")",  dgDegree[dg] ,  res[r],myDb.Path);
//         var oneJob           = new Job(jobName, solverSimple);
//         oneJob.SetControlStatement(StartupString);
//         oneJob.Activate(myBatch);
//     }
// }

// for (int dg = 0; dg < dgDegree.Length; dg++) {
//     for (int r = 0; r < res.Length; r++) {
//         string ending        = "P" + dgDegree[dg] + "K" + res[r];
//         string jobName       = string.Format("XNSEC_HEATEDCOUETTE"+ ending);
//         string StartupString = string.Format("cs:BoSSS.Application.XNSEC.FullNSEControlExamples.SteadyCouetteFlowWithTemperatureGradient({0}, {1},@\"{2}\")",  dgDegree[dg] ,  res[r],myDb.Path);
//         var oneJob           = new Job(jobName, solverXNSEC);
//         oneJob.SetControlStatement(StartupString);
//         oneJob.Activate(myBatch);
//     }
// }




In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(PollingIntervallSeconds:5);

All jobs finished.


In [ ]:
class MappedValue
{
    public double dg { get; set; }
    public double ncells { get; set; }

    public double XNSEC_TIME { get; set; }
    public double SIMPLE_TIME { get; set; }
    public double TIME_PROPORTION { get; set; }

    public int DOFs { get; set; }
    
    
}

In [ ]:
Dictionary<(int,int),double> SIMPLE_TIMES = new Dictionary<(int,int),double>();
Dictionary<(int,int),double> XNSEC_TIMES = new Dictionary<(int,int),double>();
Dictionary<(int,int),double> SIMPLE_DOFS = new Dictionary<(int,int),double>();
Dictionary<(int,int),double> XNSEC_DOFS = new Dictionary<(int,int),double>();

Dictionary<(int,int),double> PROPORTION_TIMES = new Dictionary<(int,int),double>();
Dictionary<(int,int),(double,double)> DOFS_TIME = new Dictionary<(int,int),(double,double)>();



// Dictionary<(int,int),MappedValue> DATA = new Dictionary<(int,int),MappedValue>();
List<MappedValue> dataa = new List<MappedValue>();

for (int dg = 0; dg < dgDegree.Length; dg++) {
    for (int r = 0; r < res.Length; r++) {
        string ending        = "P" + dgDegree[dg] + "K" + res[r];
        var SIMPLESessionS = myDb.Sessions.Where(s=>s.Name.Contains("SIMPLE")).Where(s=> s.Name.Contains(ending)).SingleOrDefault();
        var XNSECSessionS = myDb.Sessions.Where(s=>s.Name.Contains("XNSEC")).Where(s=> s.Name.Contains(ending)).SingleOrDefault();


        var time_seconds_simple  =  SIMPLESessionS.GetApproximateRunTime().TotalSeconds;
        var time_seconds_XNSEC  = XNSECSessionS.GetApproximateRunTime().TotalSeconds;

        SIMPLE_TIMES.Add((dg,r),time_seconds_simple);
        XNSEC_TIMES.Add((dg,r),time_seconds_XNSEC);

        PROPORTION_TIMES.Add((dg,r),time_seconds_simple/time_seconds_XNSEC);

        int dofs = SIMPLESessionS.GetDOF(VariableNames.Temperature) * 3 + SIMPLESessionS.GetDOF(VariableNames.Pressure);
        DOFS_TIME.Add((dg,r),(dofs,time_seconds_simple/time_seconds_XNSEC));


        var myvals = new MappedValue();
        myvals.XNSEC_TIME = time_seconds_XNSEC;
        myvals.SIMPLE_TIME = time_seconds_simple;
        myvals.TIME_PROPORTION =  time_seconds_simple/time_seconds_XNSEC;
        myvals.dg =dg;
        myvals.ncells = (XNSECSessionS.GetGrids()).Pick(0).NumberOfCells
        myvals.DOFs = dofs;
       
        dataa.Add(myvals);
    }
}
    

In [ ]:
var asd = dataa.OrderBy(a=>a.DOFs);

In [ ]:
asd

index,dg,ncells,XNSEC_TIME,SIMPLE_TIME,TIME_PROPORTION,DOFs
0,0,0,2.257,1.768,0.7833407177669472,160
1,1,0,3.349,3.522,1.0516572111077933,336
2,2,0,4.592,8.01,1.7443379790940767,576
3,0,1,5.173,6.468,1.250338294993234,640
4,3,0,7.573,21.419,2.828337514855407,880
5,1,1,8.211,23.662,2.881744001948605,1344
6,2,1,13.057,81.224,6.22072451558551,2304
7,3,1,15.095,257.524,17.060218615435573,3520


In [ ]:
(myDb.Sessions[0].GetGrids()).Pick(0).NumberOfCells

64

257.524